# Introduction


In [ ]:
# import libraries
import pandas as pd
import zipfile
import re
import kaggle
import datetime

In [321]:
# download dataset
!kaggle datasets download -d snehangsude/audible-dataset

^C


In [3]:
# extract csv from zip file
zipfile_name = 'audible-dataset.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [40]:
# read in csv file
audible = pd.read_csv('audible_uncleaned.csv')

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00
...,...,...,...,...,...,...,...,...
87484,Last Days of the Bus Club,Writtenby:ChrisStewart,Narratedby:ChrisStewart,7 hrs and 34 mins,09-03-17,English,Not rated yet,596.00
87485,The Alps,Writtenby:StephenO'Shea,Narratedby:RobertFass,10 hrs and 7 mins,21-02-17,English,Not rated yet,820.00
87486,The Innocents Abroad,Writtenby:MarkTwain,Narratedby:FloGibson,19 hrs and 4 mins,30-12-16,English,Not rated yet,938.00
87487,A Sentimental Journey,Writtenby:LaurenceSterne,Narratedby:AntonLesser,4 hrs and 8 mins,23-02-11,English,Not rated yet,680.00


In [17]:
audible.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB


In [18]:
audible.shape

(87489, 8)

In [45]:
# Clean author names by removing repeating string
audible['author'] = audible['author'].str.replace('Writtenby:', '')

In [46]:
# Clean narrator names by removing repeating string
audible['narrator'] = audible['narrator'].str.replace('Narratedby:', '')

In [47]:
# Convert release date to date time type
audible['releasedate'] = pd.to_datetime(audible['releasedate'])

In [42]:
# Convert duration string into minutes
for i in range(audible.time.shape[0]):
    total = 0
    # find the numeric value for hour and minute 
    time_parts = re.findall(r'(\d+) hr?|(\d+) min?', audible['time'].iloc[i])
    for part in time_parts:
        if part[0]:  # hours part
            total += int(part[0]) * 60
        if part[1]:  # minutes part
            total += int(part[1])
    audible['time'].iloc[i] = total

In [49]:
# Create new column to separate number of ratings from star ratings
audible['ratings'] = audible['stars']

In [50]:
# Extract number from ratings and store in ratings column
s = audible['ratings']
audible['ratings'] = s.str.extract(pat='(\d+) ratings')
audible['ratings'] = audible['ratings'].fillna(0)
audible['ratings'] = pd.to_numeric(audible['ratings'])

In [51]:
# Extract star rating and store in stars column
t = audible['stars']
audible['stars'] = t.str.extract(pat='(\d*\.?\d*) out of 5 stars')
audible['stars'] = audible['stars'].fillna(0)
audible['stars'] = pd.to_numeric(audible['stars'])


In [65]:
audible.language.unique()

array(['English', 'Hindi', 'spanish', 'german', 'french', 'catalan',
       'swedish', 'italian', 'danish', 'finnish', 'dutch', 'hebrew',
       'russian', 'polish', 'galician', 'afrikaans', 'icelandic',
       'romanian', 'japanese', 'tamil', 'portuguese', 'urdu', 'hungarian',
       'czech', 'bulgarian', 'mandarin_chinese', 'basque', 'korean',
       'arabic', 'greek', 'turkish', 'ukrainian', 'slovene', 'norwegian',
       'telugu', 'lithuanian'], dtype=object)

In [68]:
audible.language = audible.language.str.capitalize()

In [69]:
audible.language.unique()

array(['English', 'Hindi', 'Spanish', 'German', 'French', 'Catalan',
       'Swedish', 'Italian', 'Danish', 'Finnish', 'Dutch', 'Hebrew',
       'Russian', 'Polish', 'Galician', 'Afrikaans', 'Icelandic',
       'Romanian', 'Japanese', 'Tamil', 'Portuguese', 'Urdu', 'Hungarian',
       'Czech', 'Bulgarian', 'Mandarin_chinese', 'Basque', 'Korean',
       'Arabic', 'Greek', 'Turkish', 'Ukrainian', 'Slovene', 'Norwegian',
       'Telugu', 'Lithuanian'], dtype=object)